In [1]:
import numpy as np
import tensorflow as tf
sess = tf.Session()

C:\Users\Royce\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


RPN启发式采样

In [ ]:
def RPN_Target(gt_boxes, gt_cls, anchors, rpn_train_anchors=None):
    """
    RPN网络的目标采样(始终保持1：1的正负样本数关系)
    gt_boxes:（gt_num，（y1,x1,y2,x2,tag））
    gt_cls:（gt_num，（class_name，tag））
    anchors:（anchor_num，（y1,x1,y2,x2））
    rpn_train_anchors：训练样本数量
    """
    # 正样本
    not_padding_indices = tf.where(tf.not_equal(gt_boxes[...,-1], 0))
    gt_boxes = tf.gather(gt_boxes[...,:-1], not_padding_indices[...,0])
    gt_cls = tf.gather(gt_cls[...,0], not_padding_indices[...,0])
    # 
    iou = IOU(gt_boxes, anchors)
    # 类型Ⅰ正样本索引
    ac_pos_indices = tf.argmax(iou, axis=1)
    # 类型Ⅱ正样本索引
    anchor_iou_max = tf.reduce_max(iou, axis=0)
    anchor_iou_max_indices = tf.where(anchor_iou_max > 0.7)
    gt_iou_max_indices = tf.argmax(iou, axis=0)
    gt_iou_max_indices = tf.gather_nd(gt_iou_max_indices, anchor_iou_max_indices)
    # 合并
    ## anchors
    anchor_indices = tf.concat([ac_pos_indices, anchor_iou_max_indices[...,0]], axis=0)
    ## gts
    gts_indices = tf.concat([tf.range(tf.shape(gt_boxes)[0]), gt_iou_max_indices], axis=0)
    # 正样本与其对应的gts
    pos_ac_samples = tf.gather(anchors, anchor_indices)
    pos_gt_samples = tf.gather(gt_boxes, gts_indices)
    pos_cls = tf.gather(gt_cls, gts_indices)
    pos_deltas = Regress_Target(pos_gt_samples, pos_ac_samples)
    # 启发式采样
    pos_num = tf.minimum(tf.shape(pos_ac_samples)[0], rpn_train_anchors * 0.5)
    pos_sample_indices = tf.random_shuffle(tf.shape(pos_ac_samples)[0])[:pos_num]
    pos_ac_samples = tf.gather(pos_ac_samples, pos_sample_indices) # 目标之一
    pos_cls = tf.gather(pos_cls, pos_sample_indices) # 目标之二
    pos_deltas = tf.gather(pos_deltas, pos_sample_indices) # 目标之三
    
    # 负样本
    anchor_iou_neg_indices = tf.where(anchor_iou_max < 0.3)
    neg_ac_samples = tf.gather_nd(anchors, anchor_iou_neg_indices)
    # 启发式采样
    neg_num = tf.minimum(tf.shape(neg_ac_samples)[0], rpn_train_anchors * 0.5)
    neg_sample_indices = tf.random_shuffle(tf.shape(neg_ac_samples)[0])[:neg_num]
    neg_ac_samples = tf.gather(neg_ac_samples, neg_sample_indices)
    neg_cls = tf.zeros([neg_num])
    neg_deltas = tf.zeros([neg_num, 4])
    
    # 正、负样本合并
    ac_samples = tf.concat([pos_ac_samples, neg_ac_samples], axis=0)
    cls_samples = tf.concat([pos_cls, neg_cls], axis=0)
    deltas_samples = tf.concat([pos_deltas, neg_deltas], axis=0)
    
    # 补padding（只针对cls_samples与deltas_samples）（不足rpn_train_anchors数量的维度）
    cls_samples = tf.expand_dims(cls_samples, axis=1)
    pad_size = tf.minimum(0.0, (rpn_train_anchors-tf.shape(cls_samples)[0])/2) 
    cls_samples = tf.pad(cls_samples, [[pad_size, pad_size],[0,0]], mode="CONSTANT", constant_values=0)
    deltas_samples = tf.pad(deltas_samples, [[pad_size, pad_size],[0,0]], mode="CONSTANT", constant_values=0)
    
    # 样本分类与回归的目标最后一维、最后一列加上标签tag，0为padding，1为正样本，-1为负样本
    cls_samples = tf.pad(cls_samples, [[0, 0],[0,1]], mode="CONSTANT", constant_values=0) # 先全部初始化为0-tag
    cls_samples[pos_sample_indices, -1] = 1
    cls_samples[neg_sample_indices, -1] = -1
    
    return ac_samples, cls_samples, deltas_samples